# Machine Learning

Many people imagine that data science is mostly machine learning and that data scientists mostly build and train and tweak machine learning models all day long. (Then again, many of those people don’t actually know what machine learning is.)

In fact, data science is mostly turning business problems into data problems and collecting data and understanding data and cleaning data and formatting data, after which machine learning is almost an afterthought.

## Modeling

Before we can talk about machine learning, we need to talk about *models*.

What is a model?
- A specification of a mathematical (or probabilistic) relationship that exists between different variables. 
- In simple terms, think of models as equations that takes in certain inputs and produce certain outputs.

Examples:
- A *business model* for your social network startup may takes inputs like "number of users," "ad revenue per user," and "number of employees" and outputs your annual profit for the next several years.

- A cookbook recipe entails a model that relates inputs like "number of eaters" and "hungriness" to quantities of ingredients needed.

- In a poker game on TV, each player’s "win probability" is estimated in real time based on a model that takes into account the cards that have been revealed so far and the distribution of cards in the deck.

## What Is Machine Learning?

Everyone has his or her own exact definition, but we’ll use *machine learning* to refer to creating and using models that are *learned from data* (alternative names include *predictive modeling* or *data mining* in other context).

Typically, our goal will be to use existing data to develop models that we can use to predict various outcomes for new data, such as:
- Whether an email message is spam or not
- Whether a credit card transaction is fraudulent
- Which advertisement a shopper is most likely to click on
- Which football team is going to win the Super Bowl

Two major category of models:
1. supervised models, in which there is a set of data labeled with the correct answers to learn from.
2. unsupervised models, in which there are no such labels.

Examples:
- $height = m \times weight + b$, and we are trying to learn the best estimates of $m$ and $b$ from data
- Use a decision tree to diagnose what diseases our patients have and then use data to learn the “optimal” such tree.

## Overfitting and Underfitting

A common danger in machine learning is *overfitting* — producing a model that performs well on the data you train it on but generalizes poorly to any new data. This could involve learning *noise* in the data. Or it could involve learning to identify specific inputs rather than whatever factors are actually predictive for the desired output.

The other side of this is *underfitting*—producing a model that doesn’t perform well even on the training data, although typically when this happens you decide your model isn’t good enough and keep looking for a better one.

In the following figure, three polynomials are fit to a sample of data. (Don’t worry about how; we’ll get to that in later chapters.)

![Overfitting and Underfitting](im/fit.jpg)

Candidate Models:
- degree 0, the horizontal line that represents the mean
- degree 9, the polynomial that perfectly hits every single data point.
- degree 1, a somewhat balanced version of the previous two polynomials

What can we infer from these 3 models? Which one is the best? Which one underfits? Which one overfits?

The simplest way to prevent *overfitting* is to split the dataset, so that (for example) three-fourths of it is used to train the model, after which we measure the model’s performance on the remaining third:

In [ ]:
import random
from typing import TypeVar, List, Tuple
X = TypeVar('X')  # generic type to represent a data point

def split_data(data: List[X], prob: float) -> Tuple[List[X], List[X]]:
    """Split data into fractions [prob, 1 - prob]"""
    data = data[:]                    # Make a copy
    random.shuffle(data)              # because shuffle modifies the list.
    cut = int(len(data) * prob)       # Use prob to find a cutoff
    return data[:cut], data[cut:]     # and split the shuffled list there.

data = [n for n in range(1000)]
train, test = split_data(data, 0.75)

In [ ]:
# The proportions should be correct
#print(train)
#print(test)
print(len(train))
print(len(test))

# And the original data should be preserved (in some order)
assert sorted(train + test) == data

Often, we’ll have paired input variables and output variables. In that case, we need to make sure to put corresponding values together in either the training data or the test data:

In [ ]:
Y = TypeVar('Y')  # generic type to represent output variables

def train_test_split(xs: List[X],
                     ys: List[Y],
                     test_pct: float) -> Tuple[List[X], List[X], List[Y], List[Y]]:
    # Generate the indices and split them.
    idxs = [i for i in range(len(xs))]
    train_idxs, test_idxs = split_data(idxs, 1 - test_pct)

    return ([xs[i] for i in train_idxs],  # x_train
            [xs[i] for i in test_idxs],   # x_test
            [ys[i] for i in train_idxs],  # y_train
            [ys[i] for i in test_idxs])   # y_test

As always, we want to make sure our code works right:

In [ ]:
xs = [x for x in range(1000)]  # xs are 0 ... 999
ys = [2 * x for x in xs]       # each y_i is twice x_i
x_train, x_test, y_train, y_test = train_test_split(xs, ys, 0.25)

# Check that the proportions are correct
assert len(x_train) == len(y_train) == 750
assert len(x_test) == len(y_test) == 250

print("Training Set:")
print(x_train[:10])
print(y_train[:10])

print("Testing Set:")
print(x_test[:10])
print(y_test[:10])

If the model was overfit to the training data, then it will hopefully perform really poorly on the (completely separate) test data. Said differently, if it performs well on the test data, then you can be more confident that it’s *fitting* rather than *overfitting*.

## Correctness

Suppose I’ve come up with a cheap, noninvasive test that can be given to a newborn baby that predicts—with greater than 98% accuracy—whether the newborn will ever develop leukemia. So here's the test: *predict leukemia if and only if the baby is named Luke (which sounds sort of like "leukemia")*.

As we'll see, this test is indeed more than 98% accurate. Nonetheless, it's an incredibly stupid test, and a good illustration of why we don’t typically use "accuracy" to measure how good a (binary classification) model is.

Imagine building a model to make a binary judgment. Is this email spam? Should we hire this candidate?

Given a set of labeled data and such a predictive model, every data point lies in one of four categories:
- *True positive*: "This message is spam, and we correctly predicted spam."
- *False positive (Type 1 error)*: "This message is not spam, but we predicted spam."
- *False negative (Type 2 error)*: "This message is spam, but we predicted not spam."
- *True negative*: "This message is not spam, and we correctly predicted not spam."

We often represen these as counts in a *confusion matrix* (The name stems from the fact that the table makes it easy to see if the model is confusing two classes by mislabeling one as another):

|                    | Spam           | Not Spam       |
|:-------------------|:---------------|:---------------|
| Predict "Spam"     | True Positive  | False Positive |
| Predict "Not Spam" | False Negative | True Negative  |

Let’s see how my leukemia test fits into this framework. These days approximately 5 babies out of 1,000 are named Luke. And the lifetime prevalence of leukemia is about 1.4%, or 14 out of every 1,000 people.

If we believe these two factors are independent and apply my "Luke is for leukemia" test to 1 million people, we’d expect to see a confusion matrix like:

|            | Leukemia | No Leukemia | Total     |
|:-----------|----------|-------------|-----------|
| "Luke"     | 70       | 4,930       | 5,000     |
| Not "Luke" | 13,930   | 981,070     | 995,000   |
| Total      | 14,000   | 986,000     | 1,000,000 |

We can then use these to compute various statistics about model performance. For example, *accuracy* is defined as the fraction of correct predictions:

In [ ]:
def accuracy(tp: int, fp: int, fn: int, tn: int) -> float:
    correct = tp + tn
    total = tp + fp + fn + tn
    return correct / total

In [ ]:
print(f"accuracy = {accuracy(70, 4930, 13930, 981070)}")

That seems like a pretty impressive number. But clearly this is not a good test, which means that we probably shouldn’t put a lot of credence in raw accuracy.

It’s common to look at the combination of *precision* and *recall*. Precision measures how accurate our *positive* predictions were:

In [ ]:
def precision(tp: int, fp: int, fn: int, tn: int) -> float:
    return tp / (tp + fp)

In [ ]:
print(f"precision = {precision(70, 4930, 13930, 981070)}")

And recall measures what fraction of the positives our model identified:

In [ ]:
def recall(tp: int, fp: int, fn: int, tn: int) -> float:
    return tp / (tp + fn)

In [ ]:
print(f"recall = {recall(70, 4930, 13930, 981070)}")

These are both terrible numbers, reflecting that this is a terrible model.

Sometimes precision and recall are combined into the *F1* score, which is defined as:

In [ ]:
def f1_score(tp: int, fp: int, fn: int, tn: int) -> float:
    p = precision(tp, fp, fn, tn)
    r = recall(tp, fp, fn, tn)

    return 2 * p * r / (p + r)

In [ ]:
print(f"F1 score = {f1_score(70, 4930, 13930, 981070)}")

This is the harmonic mean of precision and recall and necessarily lies between them.

Usually the choice of a model involves a tradeoff between precision and recall.
- To improve one metric, the other metric tends to suffer. Why is this the case?

## The Bias-Variance Tradeoff

Another way of thinking about the overfitting problem is as a tradeoff between *bias* and *variance*. Both are measures of what would happen if you were to retrain your model many times on different sets of training data (from the same larger population).

Recall the figure illustrating how models may overfit or underfit data:

![Overfitting and Underfitting](im/fit.jpg)

The degree 0 model 
- a lot of mistakes for pretty much any training set (drawn from the same population), high *bias*.
- any two randomly chosen training sets should give pretty similar models (since any two randomly chosen training sets should have pretty similar average values), low *variance*.

*High bias and low variance typically correspond to underfitting*.

The degree 9 model 
- fit the training set perfectly, low *bias*.
- any two training sets would likely produce very different models, high *variance*.

*Low bias and high variance typically correspond to overfitting*.

Thinking about model problems this way can help you figure out what to do when your model doesn’t work so well.

If your model has high bias (which means it performs poorly even on your training data), one thing to try is adding more features/inputs. Going from the degree 0 model to the degree 1 model was a big improvement.

If your model has high variance, you can similarly *remove* features. But another solution is to obtain more data (if you can).

The following figure illustrates how various degree 9 polynomials are fit to sample data of different sizes.

![Overfitting and Underfitting](im/fit2.jpg)

The model fit based on 10 data points is all over the place, as we saw before.

If we instead train on 100 data points, there’s much less overfitting.

And the model trained from 1,000 data points looks very similar to the degree 1 model.

Holding model complexity constant, the more data you have, the harder it is to overfit. On the other hand, more data won’t
help with bias. If your model doesn’t use enough features to capture regularities in the data, throwing more data at it won’t help.

## Feature Extraction and Selection

When your data doesn’t have enough features, your model is likely to underfit.

When your data has too many features, it’s easy to overfit.

*Features* are whatever inputs we provide to our model.

In the simplest case, features are simply given to you. If you want to predict someone's salary based on her years of experience, then years of experience is the only feature you have.

Things become more interesting as your data becomes more complicated.

Imagine trying to build a spam filter to predict whether an email is junk or not. Most models won’t know what to do with a raw email, which is just a collection of text. You’ll have
to extract features. For example:

- Does the email contain the word *inheritance* and the $ character?
- How many times does the letter *d* appear?
- What was the domain of the sender?

The answer to a question like the first question here is simply a yes or no, which we typically encode as a 1 or 0. The second is a number. And the third is a choice from a discrete set of options.

Pretty much always, we’ll extract features from our data that fall into one of these three categories. What’s more, the types of features we have constrain the types of models we can use.
- The Naive Bayes classifier is suited to yes-or-no features, like the first one in the preceding list.
- Regression models require numeric features (which could include dummy variables that are 0s and 1s).
- decision trees can deal with numeric or categorical data.

Although in the spam filter example we looked for ways to create features, sometimes we’ll instead look for ways to remove features.

For example, your inputs might be vectors of several hundred numbers. Depending on the situation, it might be appropriate to distill these down to a handful of important dimensions and use only that small number of features. Or it might be appropriate to use a technique (e.g., regularization) that penalizes models the more features they use.

How do we choose features? That’s where a combination of experience and domain expertise comes into play. If you’ve received lots of emails, then you probably have a sense that the presence of certain words might be a good indicator of spamminess. And you might also get the sense that the number of *d*s is likely not a good indicator of spamminess. But in general this can be a trial and error process, or if feasible, do some sort of exhausive search on all possible subsets of the features.